## Bronze Schema

In [0]:
select *,_metadata.file_path from JSON.`/Volumes/gizmobox/landing/operational_data/customers/`;

create view v_bronze_customer which has all the data of customer file and also populate the file path within each row

In [0]:
create or replace view gizmobox.bronze.v_bronze_customer 
as
select *,_metadata.file_path from JSON.`/Volumes/gizmobox/landing/operational_data/customers/`;

## Silver Schema

In [0]:
select * from gizmobox.bronze.v_bronze_customer;

Databricks data profile. Run in Databricks to view.

- Remove records with null customer id
- remove records with exact duplicate records
- remove duplicate records based on created_timestamp ( using cte)

In [0]:
select * from gizmobox.bronze.v_bronze_customer where customer_id is not null order by customer_id;

In [0]:
select distinct * from gizmobox.bronze.v_bronze_customer where customer_id is not null order by customer_id;

In [0]:
with cte_max as
(select customer_id,max(created_timestamp) as created_timestamp from gizmobox.bronze.v_bronze_customer group by customer_id) 
select distinct t.* from gizmobox.bronze.v_bronze_customer t inner join cte_max c on t.customer_id=c.customer_id and c.created_timestamp = t.created_timestamp
where t.customer_id is not null 
order by t.customer_id;

Cast column to their best data type

In [0]:
with cte_max as
(select customer_id,max(created_timestamp) as created_timestamp from gizmobox.bronze.v_bronze_customer group by customer_id) 
select distinct 
  cast(t.created_timestamp as timestamp) as created_timestamp,
  t.customer_id,
  t.customer_name,
  cast(t.date_of_birth as date) as date_of_birth,
  t.email,
  cast(t.member_since as date) as member_since,
  t.telephone,
  t.file_path
from gizmobox.bronze.v_bronze_customer t 
inner join 
cte_max c 
on t.customer_id=c.customer_id and c.created_timestamp = t.created_timestamp
where t.customer_id is not null 
order by t.customer_id;

create table customers under silver schema from the above query

In [0]:
create table if not exists gizmobox.silver.customer
as
with cte_max as
(select customer_id,max(created_timestamp) as created_timestamp from gizmobox.bronze.v_bronze_customer group by customer_id) 
select distinct 
  cast(t.created_timestamp as timestamp) as created_timestamp,
  t.customer_id,
  t.customer_name,
  cast(t.date_of_birth as date) as date_of_birth,
  t.email,
  cast(t.member_since as date) as member_since,
  t.telephone,
  t.file_path
from gizmobox.bronze.v_bronze_customer t 
inner join 
cte_max c 
on t.customer_id=c.customer_id and c.created_timestamp = t.created_timestamp
where t.customer_id is not null 
order by t.customer_id;

In [0]:
select * from gizmobox.silver.customer ; 

In [0]:
describe extended gizmobox.silver.customer;